In [6]:
import numpy as np

from sklearn import datasets
from sklearn.cross_validation import train_test_split

## Iris Dataset ##

In [7]:
iris = datasets.load_iris()

print iris.feature_names
print iris.target_names

X = iris.data[:, [2, 3]]
y = iris.target

print np.unique(y)

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
['setosa' 'versicolor' 'virginica']
[0 1 2]


### Cross Validation ###
[scikit-learn documentation](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [8]:
XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=.3, random_state=0)

In [11]:
print len(X), len(y)
print len(X) * .3
print len(XTest), len(yTest)

150 150
45.0
45 45


# Perceptron #